In [ ]:
import pymysql
import pandas as pd
import getpass
import re
from collections import Counter

In [ ]:
conn = pymysql.connect(host="35.233.174.193",port=3306,
                           user=getpass.getpass("Enter username for MIMIC2 database"),
                           passwd=getpass.getpass("Enter password for MIMIC2 database"),
                           db='mimic2')

### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [ ]:
rad_data = \
pd.read_sql("""SELECT DISTINCT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 5000""",conn)
rad_data.head(5)

In [ ]:
rad_data.shape

In [ ]:
print(rad_data.iloc[20]["text"])

### Write a function that returns true or false depending on whether the report contains an impression section

#### Hints

* Not every report will have an impression section
* "INTERPRETATION:" and "CONCLUSIONS:" might be synonyms for "IMPRESSION:"

In [ ]:
def count_impression(report):
    for word in report.split():
        if word == "INTERPRETATION:" or word == "CONCLUSIONS:" or word == "IMPRESSION:":
            return True
        pass
    pass


In [ ]:
def count_impression2(report):
    headings = ["INTERPRETATION:", "CONCLUSIONS:", "IMPRESSION:"]
    for h in headings:
        if h in report:
            return True
    
    return False


In [ ]:
%timeit count_impression(rad_data.iloc[0]["text"])

In [ ]:
sum([count_impression(report) for report in rad_data["text"]])

### Write a function that returns the impression section of a report



In [ ]:
def get_impression(report):
    headings = ["INTERPRETATION:", "CONCLUSIONS:", "IMPRESSION:"]
    for h in headings:
        if h in report:
            return report[report.find(h):]
    
    return report

In [ ]:
get_impression(rad_data.iloc[20]["text"])

### Define Regular expressions for data cleansing

We have a lot of patterns that are unique and don't convey meaningful information

* De-identified names, dates, etc. 
    * `[**Clip Number (Radiology) 12569**]`
    * `DATE: [**3352-7-10**] `
    * `[**Hospital 12568**]`
    * `[**Last Name (NamePattern4) 337**]` 
    * `[**First Name8 (NamePattern2) 12565**]` 
    * `[**Last Name (NamePattern1) 12566**]`
* Separators  (e.g. `__________________`)

### Split into groups and write and test regular expressions to capture these patterns
* Write a regular expression to replace dates in the reports with ``[**DATE**]``
* Write a regular expression to replace times in the reports with ``[**TIME**]``
* Write a regular expression to replaces digits with "d", (e.g. "43 cc" would become "dd cm")

In [ ]:
date=re.compile(r"""((?P<month>[A-Z][a-z]{2,}(\.)?) (?P<day>[0-9]{1,2}))""")
last_name=re.compile(r"""(\[\*\*Last Name \((NamePattern|STitle)(\d+)?\) [0-9]*\*\*\])""")
clip = re.compile(r"""\[\*\*Clip Number \(Radiology\) \d+\*\*\]""")
date2 = re.compile(r"""DATE\: \[\*\*\d+-\d+-\d+\*\*\]""")
hospital=re.compile(r"""(\[\*\*Hospital \d+\*\*\])""")
unders = re.compile(r"""_{2,}""")

age2 = re.compile(r"""(?P<age>[0-9]+)(-|\s)y(ear(s)?|\.)(-|\s)?o(ld|\.)""")
age3 = re.compile(r"""\bage(d)? (?P<age>[0-9]+)""")
digits = re.compile(r"""\d""")
def age_in_decades(m):
    age = int(m.group("age"))
    
    return "[** Age in %ss**]"%(int(age/10)*10,)

age_in_decades(next(age2.finditer("74-year-old")))
#tmp = re.sub(age2, age_in_decades, re.sub(age3, age_in_decades, report_txt))

In [ ]:
unders.findall(rad_data.iloc[20]["text"])

In [ ]:
hospital.findall("""Dr. [**Last Name (STitle) 13    * `[**Hospital 12568**]`
927**] and Dr. [**Last Name (STitle) 13928**] on""")

In [ ]:
clip.findall("""BONE SCAN                                                       Clip # [**Clip Number (Radiology) 13926**]""")

In [ ]:
date2.findall("""DATE: [**3352-7-10**]""")